<a href="https://colab.research.google.com/github/amarildo86/datascience/blob/master/Digio_Hackathon_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>Financial Data Analytics Hackathon</center>

### Contexto do desafio

Quando os bancos digitais começaram a nascer no Brasil, realizar um bom atendimento, solucionar problemas online, ter de graça a anuidade no cartão de crédito, os saques e as transferências eram pilares de uma proposta de valor inovadora. Mais recentemente, o que antes era uma diferenciação se tornou um padrão para qualquer conta digital. Diferentes produtos atualmente são criados, como programas de fidelidade, investimentos e empréstimos instantâneos para trazer mais simplicidade e atratividade para os clientes.

Ao mesmo tempo, trocar de banco ficou tão fácil quanto começar uma nova série no canal de Streaming. Em apenas sete minutos você pode se cadastrar em um novo banco e em até 18 horas estar aprovado¹. Essa rapidez e agilidade incrementou significativamente a exposição dos clientes aos serviços financeiros como a conta, crédito e cartões.

Essa competição, cada vez mais acirrada neste novo cenário, faz com que bancos digitais tenham um grande desafio, principalmente relacionado à rentabilidade dos clientes no produto cartão de crédito. Pensando na otimização financeira deste produto, há algumas perguntas-chave:

• Qual o melhor modelo para calcular a rentabilidade destes clientes desde a sua origem?

• Entre tantas ferramentas e métodos de análise de dados, como gerar o melhor modelo financeiro de Safra de Cartões respeitando os conceitos e regras de uma demonstração de resultados (DRE), Payback e otimizando ao máximo esforço e tempo?

### Problema de negócio:
*   Previsão da linhas da DRE que compõe a Margem Operacional
*   Calcular o Payback do investimento
*   Apresentar o melhor modelo de rentabilidade do cartão desde a origem




---



In [13]:
#libs principais para análise de dados
import pandas as pd
import numpy as np

#libs para tratamento de datas/séries temporais
from datetime import datetime
from datetime import date

#lib para estatística
import statsmodels.api as sm

#libs para deep learning
import tensorflow as tf
import keras

#lib para limpar avisos desnecessários
import warnings
warnings.filterwarnings(action='ignore')

In [14]:
#lib para carregar dados do google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
#carregamento dos datasets
clientes = pd.read_csv("drive/MyDrive/digio_hackathon/clientes.csv", sep=';')
transacional = pd.read_csv("drive/MyDrive/digio_hackathon/transacional.csv", sep=';')
ml_predictions = pd.read_csv("drive/MyDrive/digio_hackathon/ml_predictions.csv", sep=';')

### 1. Descobrindo os dados
É importante lembrar que nossa tarefa é conhecer a fundo os dados, de forma que possamos *extrair conhecimento útil, extrair padrões e formular hipóteses, seguindo um processo sistematizado*.

#### 1.1 Dimensão
Saber a dimensão do conjunto de dados pode ser a primeira pista da abordagem que iremos adotar. Um conjunto de dados com apenas 1 dezena de variáveis é bem diferente de outro com algumas centenas. A quantidade de registros também pode indicar se optaremos por um modelo de Aprendizagem Profunda ou por outro de Aprendizado de Máquina puramente.

In [23]:
#retornar a dimensão dos datasets
c_obs, c_features = clientes.shape
t_obs, t_features = transacional.shape

print("Clientes: ", c_obs, "observações e", c_features, "variáveis.")
print("Transacional: ", t_obs, "observações e", t_features, "variáveis.")

Clientes:  335272 observações e 17 variáveis.
Transacional:  4923032 observações e 15 variáveis.


#### 1.2 Primeiro olhar

In [29]:
#exibição das duas primeiras linhas
clientes.head(2)

,safra,uf,nascimento,genero_biologico,renda_presumida,renda_declarada,limite_maximo,limite_global,reg_metropolitana,sistema_operacional,operadora_tim,operadora_claro,operadora_oi,operadora_vivo,operadora_nextel,operadora_others,id
0,201706,AL,1993-06-06,M,1200.0,1800.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1844674417714159308
1,201702,MT,1993-11-23,M,750.0,1444.0,750.0,750.0,NaN,iOS,0.0,1.0,0.0,0.0,0.0,0.0,691752913106883469


Agora que já demos uma primeira olhada nos dados, vamos investigar quais são os tipos de dados de cada uma das variáveis.

In [37]:
#exibir informações sobre tipo de dados
clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335272 entries, 0 to 335271
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   safra                335272 non-null  int64  
 1   uf                   335255 non-null  object 
 2   nascimento           335272 non-null  object 
 3   genero_biologico     335272 non-null  object 
 4   renda_presumida      335272 non-null  float64
 5   renda_declarada      335272 non-null  float64
 6   limite_maximo        335270 non-null  float64
 7   limite_global        335272 non-null  float64
 8   reg_metropolitana    291196 non-null  float64
 9   sistema_operacional  270767 non-null  object 
 10  operadora_tim        260599 non-null  float64
 11  operadora_claro      260599 non-null  float64
 12  operadora_oi         260599 non-null  float64
 13  operadora_vivo       260599 non-null  float64
 14  operadora_nextel     260599 non-null  float64
 15  operadora_others 

Aqui, já podemos iniciar um primeiro tratamento de dados. Observe que a variável "safra" é do tipo "inteiro". Adiante, veremos que este tipo de dado deverá ser convertido em um formato de data (datetime), afim de que seja utilizável nos modelos de séries temporais.

Agora que já olhamos os dados e também conhecemos os tipos de dados, vamos organizálos da seguinte forma: Categóricas e Quantitativas


**Variáveis Categóricas**
*   uf
*   genero_biologico
*   regiao_metropolitana
*   sistema_operacional
*   operadora_tim
*   operadora_claro
*   operadora_oi
*   operadora_vivo
*   operadora_nexthel
*   operadora_others

**Variáveis Quantitativas (Contínuas)**
*   renda_presumida
*   renda_declarada
*   limite_maximo
*   limite_global
*   limite_global

**Variáveis Quantitativas (Discretas)**
*   safra
*   id

In [30]:
#exibição das duas primeiras linhas
transacional.head(2)

,periodo,mob,spending,n_transacoes,max_atraso_mes,taxa_ocupacao_limite,flag_parcelamento_fatura,valor_fatura,custo_aquisicao,custo_manutencao,receita_spread,custo_perda_liquida,receita_servicos,custo_impostos,id
0,201905,1,0.0,0,0,0.00,0,0.0,56.4165,6.3,-0.060178,0.0,0.0,-0.002798,227261
1,201906,2,0.0,0,0,0.59,0,10.0,0.0000,6.3,0.000000,0.0,0.0,0.000000,227261
